In [3]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline
from sklearn import preprocessing
from sklearn.svm import LinearSVC
from sklearn.multiclass import OneVsOneClassifier
from sklearn import model_selection

# データファイルを読み込む
input_file = './data/income_data.txt'

# 各変数を定義しておく
Xy = []
count_class1 = 0
count_class2 = 0
max_datapoints = 25000

with open(input_file, 'r') as f:
    for line in f.readlines():
        if count_class1 >= max_datapoints and count_class2 >= max_datapoints:
            break
            
        if '?' in line:
            continue
            
        data = line[:-1].split(', ')

        if data[-1] == '<=50K' and count_class1 < max_datapoints:
            Xy.append(data)
            count_class1 += 1

        if data[-1] == '>50K' and count_class2 < max_datapoints:
            Xy.append(data)
            count_class2 += 1


Xy = np.array(Xy)
Xy

array([['39', 'State-gov', '77516', ..., '40', 'United-States', '<=50K'],
       ['50', 'Self-emp-not-inc', '83311', ..., '13', 'United-States',
        '<=50K'],
       ['38', 'Private', '215646', ..., '40', 'United-States', '<=50K'],
       ...,
       ['58', 'Private', '151910', ..., '40', 'United-States', '<=50K'],
       ['22', 'Private', '201490', ..., '20', 'United-States', '<=50K'],
       ['52', 'Self-emp-inc', '287927', ..., '40', 'United-States',
        '>50K']], dtype='<U26')

In [ ]:
label_encoder = []
Xy_encoded = np.empty(Xy.shape)
for i, item in enumerate(Xy[0]):
    if item.isdigit():
        Xy_encoded[:]